<a href="https://colab.research.google.com/github/Nid989/LLMs-Overview/blob/main/langchain/functions_tools_and_agents_with_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
%%capture
!pip install openai python-dotenv langchain tiktoken
!pip install pydantic==1.10.8
!pip install langchain[docarray]

In [2]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv(filename="./.env.txt"))
openai.api_key = os.environ['OPENAI_API_KEY']

#### `OpenAI: Function Calls`

In [32]:
import json

# Example dummy function hard coded to return the same weather
# IN production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"]
    }
    return json.dumps(weather_info)

In [10]:
# define a function (`As proposed by OpenAI`)
# `description` parameter (both) are really important, since this will be passed directly
# to the LLM and the model is gonna use this description to determine whether to use the function :)
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Fransico, CA"
                },
                "unit": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"]
                }
            },
            "required": ["location"]
        }
    }
]

In [11]:
messages = [
    {
        "role": "user",
        "content": "what's the weather like in Boston?"
    }
]

In [12]:
import openai

In [15]:
response = openai.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions
)

In [21]:
print(response)

ChatCompletion(id='chatcmpl-8NRXBU3NwwEKBLKOxUnDEcxjevmdE', choices=[Choice(finish_reason='function_call', index=0, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "location": "Boston, MA"\n}', name='get_current_weather'), tool_calls=None))], created=1700598077, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=18, prompt_tokens=84, total_tokens=102))


In [27]:
response_message = response.choices[0].message
response_message

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "location": "Boston, MA"\n}', name='get_current_weather'), tool_calls=None)

In [42]:
response_message.content # None

In [29]:
response_message.function_call

FunctionCall(arguments='{\n  "location": "Boston, MA"\n}', name='get_current_weather')

In [34]:
args = json.loads(response_message.function_call.arguments)
args

{'location': 'Boston, MA'}

In [36]:
get_current_weather(args)

'{"location": {"location": "Boston, MA"}, "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'

In [37]:
# w/ OpenAI Function Calls, it doesnot utilize the defined function to generate the response
# rather it will tell us what function to call and what the arguments to the specific function should be!

In [38]:
# what if the message passed to the OpenAI model API is not related to the already defined function at all.
messages = [
    {
        "role": "user",
        "content": "hi!"
    }
]

In [39]:
response = openai.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions
)

In [41]:
print(response) # output `message` includes not None content and no function call argument as seen in the previous response

ChatCompletion(id='chatcmpl-8NRiAXzwV24TFvTYFUKrFOqlSFVTi', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None))], created=1700598758, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=10, prompt_tokens=78, total_tokens=88))


In [44]:
# `function_call` parameter either to force function call or not
# mode: `auto`
messages = [
    {
        "role": "user",
        "content": "hi!"
    }
]

response = openai.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="auto" # default (i.e. the LLM choose b/w functions and self-response)
)
print(response)

ChatCompletion(id='chatcmpl-8NRlN8NUZmfQmVJh4q7pKU3rZ9Zd0', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None))], created=1700598957, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=10, prompt_tokens=78, total_tokens=88))


In [45]:
# mode: `none` # force LLM not make a function call
messages = [
    {
        "role": "user",
        "content": "What's the weather in Boston?"
    }
]

response = openai.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="none"
)
print(response)

ChatCompletion(id='chatcmpl-8NRmnQPmuPOHS0qbM6DE1TMTjhwTK', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='The current weather in Boston is not available.', role='assistant', function_call=None, tool_calls=None))], created=1700599045, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=9, prompt_tokens=84, total_tokens=93))


In [46]:
# mode: assign a specific function (i.e. force the model to compulsorily use the function-call w/ function name provided)
messages = [
    {
        "role": "user",
        "content": "hi!"
    }
]

response = openai.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"}
)
print(response)

ChatCompletion(id='chatcmpl-8NRohsA1OLf1tWmGtBFjf8nRVIO8O', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "location": "San Francisco, CA"\n}', name='get_current_weather'), tool_calls=None))], created=1700599163, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=12, prompt_tokens=85, total_tokens=97))


In [52]:
messages = [
    {
        "role": "user",
        "content": "What's the weather in Boston?",
    }
]

response = openai.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"}
)
print(response)

ChatCompletion(id='chatcmpl-8NRyYjEgE66WrfuV730Bpgvs8ofAD', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "location": "Boston, MA"\n}', name='get_current_weather'), tool_calls=None))], created=1700599774, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=11, prompt_tokens=90, total_tokens=101))


In [49]:
# NOTE: under the `usage` output parameter, we can see that w/ and w/o `functions` as input-argument
# the prompt_tokens utilization differs drastically, from 85 (w/ function-calling) to 15 (w/o function calling)
# since extra tokens are utilized to describe the function-calling procedure to the OpenAI LLM API.

# This can put constraint, since we have a limit over the OpenAI `prompt_tokens`, thus now we do have to keep
# track of the total input tokens already used by function-calling proc. alongwith the messages supplied

In [56]:
messages = [
    {
        "role": "user",
        "content": "What's the weather in Boston?"
    }
]

response = openai.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="auto"
)
print(response)

ChatCompletion(id='chatcmpl-8NS06f41dvims8pNYxkI8ler59GdU', choices=[Choice(finish_reason='function_call', index=0, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "location": "Boston, MA"\n}', name='get_current_weather'), tool_calls=None))], created=1700599870, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=18, prompt_tokens=83, total_tokens=101))


In [62]:
messages.append(response.choices[0].message) # appending to the list of messages to simulate calling to function w/ args

In [63]:
args = json.loads(response.choices[0].message.function_call.arguments)
observation = get_current_weather(args)

In [68]:
messages.append(
    {
        "role": "function",
        "name": "get_current_weather",
        "content": observation
    }
)
print(messages)

[{'role': 'user', 'content': "What's the weather in Boston?"}, ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "location": "Boston, MA"\n}', name='get_current_weather'), tool_calls=None), ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "location": "Boston, MA"\n}', name='get_current_weather'), tool_calls=None), {'role': 'function', 'name': 'get_current_weather', 'content': '{"location": {"location": "Boston, MA"}, "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'}, {'role': 'function', 'name': 'get_current_weather', 'content': '{"location": {"location": "Boston, MA"}, "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'}]


In [67]:
response = openai.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages
)
print(response)

ChatCompletion(id='chatcmpl-8NS35jnze9q3iVfQ70qUnl2wKXf7i', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='The current weather in Boston is 72°F with sunny and windy conditions.', role='assistant', function_call=None, tool_calls=None))], created=1700600055, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=15, prompt_tokens=97, total_tokens=112))


#### `LangChain Expression Language (LECL)`

In [3]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser  # will take a ChatMessage and convert to string

##### `Simple Chain`

In [4]:
prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

In [5]:
chain = prompt | model | output_parser

In [6]:
chain.invoke({"topic": "bears"})

"Why don't bears like fast food?\nBecause they can't catch it!"

##### `More Complex Chain and Runnnable Map (to supply user-provided inputs to prompt)`

In [10]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

In [11]:
vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

In [12]:
retriever.get_relevant_documents("Where did harrison work?")

[Document(page_content='harrison worked at kensho'),
 Document(page_content='bears like to eat honey')]

In [13]:
retriever.get_relevant_documents("What do bears like to eat?")

[Document(page_content='bears like to eat honey'),
 Document(page_content='harrison worked at kensho')]

In [14]:
template = """Answer the question based only on the following context:
{context}

Question: {question}"""
prompt = ChatPromptTemplate.from_template(template)

In [15]:
# How are we going to build the chain?
# i. We want the first and only input to the chain to be the user question.
# ii. Then we want to fetch the relevant document for the context which will be passed to the prompt
# iii. Finally, we want to pass the formulated prompt to the model and the output-parser later on.

In [16]:
# i. we want to create `Something` which takes a single question and turns it into
# a dictionary of 2 elements, `context` and `question`.
from langchain.schema.runnable import RunnableMap

In [17]:
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

In [18]:
chain.invoke({"question": "Where did harrison work?"})

'Harrison worked at Kensho.'

In [19]:
inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})

In [20]:
inputs.invoke({"question": "Where did harrison work?"})

{'context': [Document(page_content='harrison worked at kensho'),
  Document(page_content='bears like to eat honey')],
 'question': 'Where did harrison work?'}

##### `Bind and OpenAI Functions`

In [22]:
functions = [
    {
        "name": "weather_search",
        "description": "Search for weather given at airport code",
        "parameters": {
            "type": "object",
            "properties": {
                "airport_code": {
                    "type": "string",
                    "description": "The airport code to get the weather for"
                },
            },
            "required": ["airport_code"]
        }
    }
]

In [23]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}")
    ]
)
model = ChatOpenAI(temperature=0).bind(functions=functions)

In [24]:
runnable = prompt | model

In [25]:
runnable.invoke({"input": "what is the weather in sf?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "airport_code": "SFO"\n}', 'name': 'weather_search'}})

In [26]:
functions = [
    {
        "name": "weather_search",
        "description": "Search for weather given at airport code",
        "parameters": {
            "type": "object",
            "properties": {
                "airport_code": {
                    "type": "string",
                    "description": "The airport code to get the weather for"
                },
            },
            "required": ["airport_code"]
        }
    },
    {
        "name": "sports_search",
        "description": "Search for news of recent sport events",
        "parameters": {
            "type": "object",
            "properties": {
                "team_name": {
                    "type": "string",
                    "description": "The sports team to search for"
                },
            },
            "required": ["team_name"]
        }
    }
]

In [27]:
model = model.bind(functions=functions)

In [29]:
runnable = prompt | model

In [30]:
runnable.invoke({"input": "how did the patriots do yesterday?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "team_name": "patriots"\n}', 'name': 'sports_search'}})

##### `Fallbacks`
`with LangChain, we can attach fallbacks not only to the model but any component of the chain or the chain as whole.`

In [32]:
# let's say we want to output a JSON formatted string from the LLM
# NOTE: using an older version of OpenAI LLM API just to showcase the need for `fallbacks`.
from langchain.llms import OpenAI
import json

In [33]:
simple_model = OpenAI(
    temperature=0,
    max_tokens=1000,
    model="text-davinci-001"
)
simple_chain = simple_model | json.loads

In [34]:
challenge = "write three poems in a json blob, where each poem is a json blob of a title, author, and first line"

In [35]:
simple_model.invoke(challenge)

'\n\n["The Waste Land","T.S. Eliot","April is the cruelest month, breeding lilacs out of the dead land"]\n\n["The Raven","Edgar Allan Poe","Once upon a midnight dreary, while I pondered, weak and weary"]\n\n["Ode to a Nightingale","John Keats","Thou still unravish\'d bride of quietness, Thou foster-child of silence and slow time"]'

In [37]:
# calling the below line, we get the JSONDecodeError, since the output string
# eventhough it has some structure, cannot be decoded by json.loads() function.
# simple_chain.invoke(challenge)

In [38]:
model = ChatOpenAI(temperature=0) # using a newer version of OpenAI models
chain = model | StrOutputParser() | json.loads

In [39]:
chain.invoke(challenge)

{'poem1': {'title': 'Whispers of the Wind',
  'author': 'Emily Rivers',
  'first_line': 'Softly it comes, the whisper of the wind'},
 'poem2': {'title': 'Silent Serenade',
  'author': 'Jacob Moore',
  'first_line': 'In the stillness of night, a silent serenade'},
 'poem3': {'title': 'Dancing Shadows',
  'author': 'Sophia Anderson',
  'first_line': 'Shadows dance upon the moonlit floor'}}

In [40]:
# so we can start with a simple chain and attach fallbacks with `.with_fallbacks`
# so first the core runnable `simple_chain` will try to run and if error is raised
# then it goes through list of runnables provided as fallbacks, in this case its just
# 1 extra runnable `chain`.
final_chain = simple_chain.with_fallbacks([chain])

In [41]:
final_chain.invoke(challenge)

{'poem1': {'title': 'Whispers of the Wind',
  'author': 'Emily Rivers',
  'first_line': 'Softly it comes, the whisper of the wind'},
 'poem2': {'title': 'Silent Serenade',
  'author': 'Jacob Moore',
  'first_line': 'In the stillness of night, a silent serenade'},
 'poem3': {'title': 'Dancing Shadows',
  'author': 'Sophia Anderson',
  'first_line': 'Shadows dance upon the moonlit floor'}}

In [42]:
import langchain
langchain.debug = True
final_chain.invoke(challenge)

[chain/start] [1:chain:RunnableWithFallbacks] Entering Chain run with input:
{
  "input": "write three poems in a json blob, where each poem is a json blob of a title, author, and first line"
}
[chain/start] [1:chain:RunnableWithFallbacks > 2:chain:RunnableSequence] Entering Chain run with input:
{
  "input": "write three poems in a json blob, where each poem is a json blob of a title, author, and first line"
}
[llm/start] [1:chain:RunnableWithFallbacks > 2:chain:RunnableSequence > 3:llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "write three poems in a json blob, where each poem is a json blob of a title, author, and first line"
  ]
}
[llm/end] [1:chain:RunnableWithFallbacks > 2:chain:RunnableSequence > 3:llm:OpenAI] [1.32s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "\n\n[\"The Waste Land\",\"T.S. Eliot\",\"April is the cruelest month, breeding lilacs out of the dead land\"]\n\n[\"The Raven\",\"Edgar Allan Poe\",\"Once upon a midnig

{'poem1': {'title': 'Whispers of the Wind',
  'author': 'Emily Rivers',
  'first_line': 'Softly it comes, the whisper of the wind'},
 'poem2': {'title': 'Silent Serenade',
  'author': 'Jacob Moore',
  'first_line': 'In the stillness of night, a silent serenade'},
 'poem3': {'title': 'Dancing Shadows',
  'author': 'Sophia Anderson',
  'first_line': 'Shadows dance upon the moonlit floor'}}

In [45]:
langchain.debug = False

##### `Interface`

In [43]:
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [47]:
# We will explore few different elements of the interface

In [48]:
chain.invoke({"topic": "bears"}) # synchronous method that calls it on 1 single input.

"Why don't bears wear shoes? \n\nBecause they already have bear feet!"

In [49]:
chain.batch([{"topic": "bears"}, {"topic": "frogs"}]) # synchronous method that calls it on list of inputs.

["Why don't bears wear shoes?\n\nBecause they have bear feet!",
 'Why are frogs so happy?\n\nBecause they eat whatever bugs them!']

In [51]:
# stream back the output based on tokens generated (ideal for showcasing a streaming property when recording bot-replies in chat-interface)
for t in chain.stream({"topic": "bears"}):
    print(t)


Why
 don
't
 bears
 wear
 socks
?
 


Because
 they
 have
 bear
 feet
!



In [52]:
# calls it in a asynchronous manner
response = await chain.ainvoke({"topic": "bears"})
response

"Why don't bears wear shoes? \n\nBecause they have bear feet!"

#### `OpenAI Function Calling in LangChain`

In [54]:
# Objective: How to use OpenAI functions and combine them with LCEL (LangChain Expression Language)
# Additionally, explore pydantic which makes it easier to work with OpenAI functions.

In [55]:
from typing import List
from pydantic import BaseModel, Field

##### `Pydantic Syntax`
`Pydantic data classes are a blend of Python's data classes with the validation power of pydantic (since in normal python we specify data-types for each attribute but they're not validated and are just used for user reference)`

`They offer a concise way to define data structures while ensuring that the data adheres to specified types and constraints.`

In [58]:
# in standard python you would create a class like this:
class User:
    def __init__(self, name: str, age: int, email: str):
        self.name = name
        self.age = age
        self.email = email

In [59]:
foo = User(name="Joe", age=32, email="joe@gmail.com")
foo.name

'Joe'

In [60]:
foo = User(name="Joe", age="bar", email="joe@gmail.com")
foo.age # expected data-type for attr. `age` was "int" but the class cannot validate and accepted "str" too.

'bar'

In [61]:
class pUser(BaseModel):
    name: str
    age: int
    email: str

In [63]:
foo_p = pUser(name="Joe", age=32, email="joe@gmail.com")
foo_p.name

'Joe'

In [65]:
# Note: below code will give ValidationError, since attr. `age` is supposed to be "int"
# foo_p = pUser(name="Joe", age="bar", email="joe@gmail.com")

In [67]:
# with Pydantic we can nest these data-structures
class Class(BaseModel):
    students: List[pUser]

In [68]:
obj = Class(
    students=[pUser(name="Joe", age=32, email="joe@gmail.com")]
)

In [69]:
obj

Class(students=[pUser(name='Joe', age=32, email='joe@gmail.com')])

##### `Pydantic to OpenAI function definition`

In [70]:
class WeatherSearch(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str = Field(description="airport code to get weather for")

In [71]:
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

In [72]:
weather_function = convert_pydantic_to_openai_function(WeatherSearch)

In [74]:
weather_function # the main description is pulled from the DocString of the class definition

{'name': 'WeatherSearch',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'title': 'WeatherSearch',
  'description': 'Call this with an airport code to get the weather at that airport',
  'type': 'object',
  'properties': {'airport_code': {'title': 'Airport Code',
    'description': 'airport code to get weather for',
    'type': 'string'}},
  'required': ['airport_code']}}

In [75]:
class WeatherSearch(BaseModel):
    airport_code: str = Field(description="airport code to get weather for")

In [77]:
# DocString is a required **compulsory when defining a class definition inherited from Pydantic BaseModel
# Note: the below code will produce error a `KeyError` on calling `convert_pydantic_to_openai_function`
# since the DocString is not provided
# convert_pydantic_to_openai_function(WeatherSearch)

In [79]:
# However, when defining the attributes it is not compulsory to provide the description
class WeatherSearch2(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str

In [80]:
convert_pydantic_to_openai_function(WeatherSearch2)

{'name': 'WeatherSearch2',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'title': 'WeatherSearch2',
  'description': 'Call this with an airport code to get the weather at that airport',
  'type': 'object',
  'properties': {'airport_code': {'title': 'Airport Code', 'type': 'string'}},
  'required': ['airport_code']}}

In [82]:
# combining OpenAI function with LCEL
from langchain.chat_models import ChatOpenAI

In [83]:
model = ChatOpenAI()

In [84]:
model.invoke("what is the weather in SF today?", functions=[weather_function]) # w/ keyword args `functions`

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "airport_code": "SFO"\n}', 'name': 'WeatherSearch'}})

In [85]:
# we can bind the function invocation to the model,
# reason: we can pass that {model, function} pair together and
# do not have to worry about the explicit mentioning of kwargs
model_with_function = model.bind(functions=[weather_function])

In [86]:
model_with_function.invoke("what is the weather in SF?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "airport_code": "SFO"\n}', 'name': 'WeatherSearch'}})

##### `Forcing it to use a function`

In [87]:
model_with_forced_function = model.bind(functions=[weather_function], function_call={"name": "WeatherSearch"})

In [89]:
model_with_forced_function.invoke("what is the weather in SF?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "airport_code": "SFO"\n}', 'name': 'WeatherSearch'}})

In [90]:
model_with_forced_function.invoke("hi!")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "airport_code": "LAX"\n}', 'name': 'WeatherSearch'}})

##### `Using in a Chain`

In [91]:
from langchain.prompts import ChatPromptTemplate

In [92]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("user", "{input}")
])

In [93]:
chain = prompt | model_with_function

In [95]:
chain.invoke({"input": "what is the weather in SF?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "airport_code": "SFO"\n}', 'name': 'WeatherSearch'}})

##### `Using multiple functions`

In [96]:
# we can pass a set of functions and let the LLM decide which to use based on the question context.

In [99]:
class WeatherSearch(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str = Field(description="airport code to get weather for")

class ArtistSearch(BaseModel):
    """Call this to get the names of songs by a particular artist"""
    artist_name: str = Field(description="name of the artisit to look up")
    n: int = Field(description="number of results")

In [100]:
functions = [
    convert_pydantic_to_openai_function(WeatherSearch),
    convert_pydantic_to_openai_function(ArtistSearch)
]

In [101]:
model_with_functions = model.bind(functions=functions)

In [102]:
model_with_functions.invoke("what is the weather in SF?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n"airport_code": "SFO"\n}', 'name': 'WeatherSearch'}})

In [103]:
model_with_functions.invoke("what are three songs by taylor swift?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n"artist_name": "taylor swift",\n"n": 3\n}', 'name': 'ArtistSearch'}})

In [104]:
model_with_functions.invoke("hi!")

AIMessage(content='Hello! How can I assist you today?')

#### `Tagging and Extraction`
`Allows to extract structured data from unstructured text`

In [105]:
from typing import List
from pydantic import BaseModel, Field
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

##### `Tagging`

In [108]:
class Tagging(BaseModel):
    """Tag the piece of text with particular info."""
    sentiment: str = Field(description="sentiment of text, should be `pos`, `neg` or `neutral`") # through the tags (i.e. `pos`, `neg`, `neutral`) we're telling the LLM how the data should be structured,
    language: str = Field(description="language of text (should be ISO 639-1 code)")

In [109]:
convert_pydantic_to_openai_function(Tagging)

{'name': 'Tagging',
 'description': 'Tag the piece of text with particular info.',
 'parameters': {'title': 'Tagging',
  'description': 'Tag the piece of text with particular info.',
  'type': 'object',
  'properties': {'sentiment': {'title': 'Sentiment',
    'description': 'sentiment of text, should be `pos`, `neg` or `neutral`',
    'type': 'string'},
   'language': {'title': 'Language',
    'description': 'language of text (should be ISO 639-1 code)',
    'type': 'string'}},
  'required': ['sentiment', 'language']}}

In [110]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

In [111]:
model = ChatOpenAI(temperature=0.)

In [112]:
tagging_functions = [convert_pydantic_to_openai_function(Tagging)]

In [113]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Think carefully, and then tag the text as instructed"),
    ("user", "{input}")
])

In [114]:
model_with_functions = model.bind(
    functions=tagging_functions,
    function_call={"name": "Tagging"}
)

In [115]:
tagging_chain = prompt | model_with_functions

In [116]:
tagging_chain.invoke({"input": "I love langchain"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "sentiment": "pos",\n  "language": "en"\n}', 'name': 'Tagging'}})

In [117]:
tagging_chain.invoke({"input": "no mi piace questo cibo"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "sentiment": "neg",\n  "language": "it"\n}', 'name': 'Tagging'}})

In [118]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

In [119]:
tagging_chain = prompt | model_with_functions | JsonOutputFunctionsParser()

In [120]:
tagging_chain.invoke({"input": "non mi piace questo cibo"})

{'sentiment': 'neg', 'language': 'it'}

##### `Extraction`
`Extraction is similar to tagging, but used for extracting multiple piece of information`

In [123]:
from typing import Optional
class Person(BaseModel):
    """Information about a Person."""
    name: str = Field(description="person's name")
    age: Optional[int] = Field(description="person's age")

In [124]:
class Information(BaseModel):
    """Information to extract"""
    people: List[Person] = Field(description="List of info about people")

In [125]:
convert_pydantic_to_openai_function(Information)

{'name': 'Information',
 'description': 'Information to extract',
 'parameters': {'title': 'Information',
  'description': 'Information to extract',
  'type': 'object',
  'properties': {'people': {'title': 'People',
    'description': 'List of info about people',
    'type': 'array',
    'items': {'title': 'Person',
     'description': 'Information about a Person.',
     'type': 'object',
     'properties': {'name': {'title': 'Name',
       'description': "person's name",
       'type': 'string'},
      'age': {'title': 'Age',
       'description': "person's age",
       'type': 'integer'}},
     'required': ['name']}}},
  'required': ['people']}}

In [126]:
extraction_functions = [convert_pydantic_to_openai_function(Information)]
extraction_model = model.bind(functions=extraction_functions, function_call={"name": "Information"})

In [128]:
extraction_model.invoke("Joe is 30, his mom is Martha") # model seems to think when it doenot know a person's age, then "age=0", but we can do better!

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "people": [\n    {\n      "name": "Joe",\n      "age": 30\n    },\n    {\n      "name": "Martha",\n      "age": 0\n    }\n  ]\n}', 'name': 'Information'}})

In [129]:
# Note: below prompt makes sure via `system-prior` that when a particular
# attribute value is unknown, then ignore that attribute for that instance.
prompt = ChatPromptTemplate.from_messages([
    ("system", "Extract the relevant information, if not explicitly provided do not guess. Extract partial info"),
    ("user", "{input}")
])

In [130]:
extraction_chain = prompt | extraction_model

In [131]:
extraction_chain.invoke({"input": "Joe is 30, his mom is Martha"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "people": [\n    {\n      "name": "Joe",\n      "age": 30\n    },\n    {\n      "name": "Martha"\n    }\n  ]\n}', 'name': 'Information'}})

In [132]:
extraction_chain = prompt | extraction_model | JsonOutputFunctionsParser()

In [136]:
# Note: considering from the standpoint of `Extraction procedure`, we do not require
# the `people` item to be specified, what we want is just a list of dictionaries
# constituting the function calls
extraction_chain.invoke({"input": "Joe is 30, his mom is Martha"})

{'people': [{'name': 'Joe', 'age': 30}, {'name': 'Martha'}]}

In [142]:
# looks for particular key in the output and extract only that
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

In [141]:
extraction_chain = prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name="people")

In [140]:
extraction_chain.invoke({"input": "Joe is 30, his mom is Martha"})

[{'name': 'Joe', 'age': 30}, {'name': 'Martha'}]

##### `Doing it for real`
`We can apply tagging to a larger body of text.`
`For example, let's load this blog post and extract tag information from a sub-set of the text`

In [144]:
from langchain.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
documents = loader.load()

In [145]:
doc = documents[0]

In [147]:
page_content = doc.page_content[:10000]
print(page_content[:1000])







LLM Powered Autonomous Agents | Lil'Log







































Lil'Log






















Posts




Archive




Search




Tags




FAQ




emojisearch.app









      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


 


Table of Contents



Agent System Overview

Component One: Planning

Task Decomposition

Self-Reflection


Component Two: Memory

Types of Memory

Maximum Inner Product Search (MIPS)


Component Three: Tool Use

Case Studies

Scientific Discovery Agent

Generative Agents Simulation

Proof-of-Concept Examples


Challenges

Citation

References





Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general

In [148]:
class Overview(BaseModel):
    """Overview of a section of text."""
    summary: str = Field(description="Provide a concise summary of the content.")
    language: str = Field(description="Provide the language that the content is written in.")
    keywords: str = Field(description="Provide keywords related to the content.")

In [151]:
overview_tagging_functions = [
    convert_pydantic_to_openai_function(Overview)
]
tagging_model = model.bind(
    functions=overview_tagging_functions,
    function_call={"name": "Overview"}
)
tagging_chain = prompt | tagging_model | JsonOutputFunctionsParser()

In [152]:
tagging_chain.invoke({"input": page_content})

{'summary': 'This article discusses the concept of building autonomous agents powered by LLM (large language model) as their core controller. It explores the key components of such agent systems, including planning, memory, and tool use. It also covers various techniques for task decomposition and self-reflection in autonomous agents. The article provides examples of case studies and challenges in implementing LLM-powered agents.',
 'language': 'English',
 'keywords': 'LLM, autonomous agents, planning, memory, tool use, task decomposition, self-reflection, case studies, challenges'}

In [153]:
class Paper(BaseModel):
    """Information about papers mentioned."""
    title: str
    author: Optional[str]

class Info(BaseModel):
    """Information to extract."""
    papers: List[Paper]

In [155]:
paper_extraction_function = [
    convert_pydantic_to_openai_function(Info)
]
extraction_model = model.bind(
    functions=paper_extraction_function,
    function_call={"name": "Info"}
)
extraction_chain = prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name="papers")

In [158]:
extraction_chain.invoke({"input": page_content})
# this is actually title we're passing in, we don't want that, it should rather extract the papers mentioned within.

[{'title': 'LLM Powered Autonomous Agents', 'author': 'Lilian Weng'}]

In [156]:
template = """A article will be passed to you. Extract from it all papers that are mentioned by this article.

Do not extract the name of the article itself. If no papers are mentioned that's fine - you don't need to extract any! Just return an empty list.

Do not make up or guess ANY extra information. Only extract what exactly is in the text."""

prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", "{input}")
])

In [159]:
extraction_chain = prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name="papers")

In [161]:
extraction_chain.invoke({"input": page_content})

[{'title': 'Chain of thought (CoT; Wei et al. 2022)', 'author': 'Wei et al.'},
 {'title': 'Tree of Thoughts (Yao et al. 2023)', 'author': 'Yao et al.'},
 {'title': 'LLM+P (Liu et al. 2023)', 'author': 'Liu et al.'},
 {'title': 'ReAct (Yao et al. 2023)', 'author': 'Yao et al.'},
 {'title': 'Reflexion (Shinn & Labash 2023)', 'author': 'Shinn & Labash'},
 {'title': 'Chain of Hindsight (CoH; Liu et al. 2023)',
  'author': 'Liu et al.'},
 {'title': 'Algorithm Distillation (AD; Laskin et al. 2023)',
  'author': 'Laskin et al.'}]

In [163]:
extraction_chain.invoke({"input": "hi"}) # based on the instruction from prompt `return empty list`

[]

In [164]:
# what is we want to extract all the papers mentioned in the whole article.
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_overlap=0)

In [165]:
splits = text_splitter.split_text(doc.page_content)

In [166]:
len(splits)

14

In [167]:
# Given the above proc., we are gonna try to automatize the above proc. + extraction + output-parsing
# i. We're gonna take the doc.page_content and split it up into chunks
# ii. pass individual split to the extraction chain defined above
# iii. finally, join all the results

In [168]:
def flatten(matrix):
    flat_list = []
    for row in matrix:
        flat_list += row
    return flat_list

In [169]:
flatten([[1, 2], [3, 4]])

[1, 2, 3, 4]

In [175]:
# i. (a) we need to create some-method of split the page-content and passing it into the chain.
# for that we require each split (i.e. a string chunk) to be fed as dictionary with the "input" key-value
from langchain.schema.runnable import RunnableLambda

In [176]:
# RunnableLambda is simple wrapper in LangChain that takes in a lambda function and converts to Runnable object
prep = RunnableLambda(
    lambda x: [{"input": doc_chunk} for doc_chunk in text_splitter.split_text(x)]
)

In [177]:
prep.invoke("hi")

[{'input': 'hi'}]

In [178]:
# NOTE: extraction_chain operates on single, and here we have list of elements,
# .map() -> take previous input (i.e. list) and map the chain (i.e. extraction_chain) over them.
# Additionally, since `flatten` is not the first component so not compulsory to wrap it into a
# RunnableLambda (but we can its optional, works either way)
chain = prep | extraction_chain.map() | flatten

In [179]:
chain.invoke(doc.page_content)

[{'title': 'AutoGPT', 'author': ''},
 {'title': 'GPT-Engineer', 'author': ''},
 {'title': 'BabyAGI', 'author': ''},
 {'title': 'Chain of thought (CoT; Wei et al. 2022)', 'author': 'Wei et al.'},
 {'title': 'Tree of Thoughts (Yao et al. 2023)', 'author': 'Yao et al.'},
 {'title': 'LLM+P (Liu et al. 2023)', 'author': 'Liu et al.'},
 {'title': 'ReAct (Yao et al. 2023)', 'author': 'Yao et al.'},
 {'title': 'Reflexion (Shinn & Labash 2023)', 'author': 'Shinn & Labash'},
 {'title': 'Reflexion framework', 'author': 'Shinn & Labash'},
 {'title': 'Chain of Hindsight', 'author': 'Liu et al.'},
 {'title': 'Algorithm Distillation', 'author': 'Laskin et al.'},
 {'title': 'Algorithm Distillation', 'author': 'Laskin et al. 2023'},
 {'title': 'ED (expert distillation)', 'author': ''},
 {'title': 'RL^2', 'author': 'Duan et al. 2017'},
 {'title': 'LSH: Locality-Sensitive Hashing', 'author': ''},
 {'title': 'ANNOY: Approximate Nearest Neighbors Oh Yeah', 'author': ''},
 {'title': 'HNSW: Hierarchical Navi

#### `Tools and Routing`